In [ ]:
# Multiplicative scaling constants
B_b   = 1e1
b_B   = 1/B_b
G_1   = 1e9
one_G = 1/G_1
G_K   = 1e6
K_G   = 1/G_K
G_M   = 1e3
M_G   = 1/G_M
K_1   = 1e3
one_K = 1/K_1
M_1   = 1e6
one_M = 1/M_1
m_1   = 1e-3
one_m = 1/m_1
m_u   = 1e3 
u_m   = 1/m_u
u_1   = 1e-6
one_u = 1/u_1
n_1   = 1e-9
one_n = 1/n_1
n_m   = 1e-6
m_n   = 1/n_m
n_u   = 1e-3
u_n   = 1/n_u

sz1K  = 1024
sz1M  = sz1K*sz1K
sz1G  = sz1M*sz1K


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis
import re
from datetime import datetime

sns.set(style="darkgrid")

# Compute statistical moments: mean, std, skewness, kurtosis
def compute_moments(series):
    values = series #.dropna()
    return {
        'count': len(values),
        'mean': values.mean(),
        'std': values.std(),
        'skew': skew(values),
        'kurtosis': kurtosis(values)
    }

# Load the log file into a list of lines
def load_log_file(path):
    with open(path, 'r') as f:
        lines = f.readlines()
    return [line.strip() for line in lines if line.strip()]


In [ ]:
pip install pyyaml

In [ ]:
import yaml
import pandas as pd
from pandas import json_normalize

# Load YAML
with open("cpu_sim.yaml", "r") as f:
    data = yaml.safe_load(f)

# Flatten into a DataFrame
prmtrs_df = json_normalize(data, sep=".")
print(prmtrs_df.T)  # transpose to make it easier to read

cmp_ltnc_nS_B   = float(prmtrs_df['cmp_ltnc_nS_B'].iloc[0])
output_size_GB  = float(prmtrs_df['output_size_GB'].iloc[0])
nic_Gbps        = float(prmtrs_df['nic_Gbps'].iloc[0])
frame_sz_MB     = float(prmtrs_df['frame_sz_MB'].iloc[0])
frame_cnt       = int(prmtrs_df['frame_cnt'].iloc[0])
cmpnt_cnt       = int(prmtrs_df['cmpnt_cnt'].iloc[0])
avg_bit_rt_Gbps = float(prmtrs_df['avg_bit_rt_Gbps'].iloc[0])

In [ ]:
prmtrs_df

In [ ]:
print(cmp_ltnc_nS_B,output_size_GB,nic_Gbps,frame_sz_MB,frame_cnt,cmpnt_cnt,avg_bit_rt_Gbps)

In [ ]:
# Network Latency Samples
import numpy as np
import matplotlib.pyplot as plt

nl_uS = float(one_u*M_1*frame_sz_MB/(b_B*G_1*nic_Gbps))
# Target mean and std
nl_mean_uS = nl_uS
nl_std_uS = nl_uS/3

# Derive Gamma distribution parameters
nl_k = (nl_mean_uS / nl_std_uS) ** 2         # shape
nl_theta = (nl_std_uS ** 2) / nl_mean_uS     # scale

# Sample from Gamma
raw_samples = np.random.gamma(shape=nl_k, scale=nl_theta, size=1+frame_cnt*cmpnt_cnt)
# Bound samples to lower bound
nl_smpls_uS = np.clip(raw_samples, nl_mean_uS, None)

# Verification
print("Empirical mean:", np.mean(nl_smpls_uS))
print("Empirical std:",  np.std(nl_smpls_uS))

# Plot histogram
plt.figure(figsize=(8, 5))
plt.hist(nl_smpls_uS, bins=50, density=True, alpha=0.7, color="blue", edgecolor="black")
plt.title(f"Clipped Gamma Distribution Samples (mean={nl_mean_uS}, std={nl_std_uS})")
plt.xlabel("Value")
plt.ylabel("Density")
plt.grid(True, linestyle="--", alpha=0.6)
plt.show()


In [ ]:
print(nl_mean_uS, nl_std_uS, nl_k, nl_theta)

In [ ]:
# Frame Size Samples
import numpy as np
import matplotlib.pyplot as plt

fs_mean_B = float(M_1*frame_sz_MB)
# Target mean and std
fs_std_B = 0.1*fs_mean_B

# Derive Gamma distribution parameters
fs_k = (fs_mean_B / fs_std_B) ** 2         # shape
fs_theta = (fs_std_B ** 2) / fs_mean_B     # scale

# Sample from Gamma
fs_smpls_B = np.random.gamma(shape=fs_k, scale=fs_theta, size=frame_cnt)

# Verification
print("Empirical mean:", np.mean(fs_smpls_B))
print("Empirical std:",  np.std(fs_smpls_B))

# Plot histogram
plt.figure(figsize=(8, 5))
plt.hist(fs_smpls_B, bins=50, density=True, alpha=0.7, color="blue", edgecolor="black")
plt.title(f"Gamma Distribution Samples (mean={fs_mean_B}, std={fs_std_B})")
plt.xlabel("Value")
plt.ylabel("Density")
plt.grid(True, linestyle="--", alpha=0.6)
plt.show()


In [ ]:
print(fs_mean_B, fs_std_B, fs_k, fs_theta)

In [ ]:
# Out Size Latency Samples
import numpy as np
import matplotlib.pyplot as plt

os_mean_B = float(G_1*prmtrs_df['output_size_GB'])
# Target mean and std
os_std_B = 0.1*os_mean_B

# Derive Gamma distribution parameters
os_k = (os_mean_B / os_std_B) ** 2         # shape
os_theta = (os_std_B ** 2) / os_mean_B     # scale

# Sample from Gamma
os_smpls_B = np.random.gamma(shape=os_k, scale=os_theta, size=1+frame_cnt*cmpnt_cnt)

# Verification
print("Empirical mean:", np.mean(os_smpls_B))
print("Empirical std:",  np.std(os_smpls_B))

# Plot histogram
plt.figure(figsize=(8, 5))
plt.hist(os_smpls_B, bins=50, density=True, alpha=0.7, color="blue", edgecolor="black")
plt.title(f"Gamma Distribution Samples (mean={os_mean_B}, std={os_std_B})")
plt.xlabel("Value")
plt.ylabel("Density")
plt.grid(True, linestyle="--", alpha=0.6)
plt.show()


In [ ]:
print(os_mean_B, os_std_B, os_k, os_theta)

In [ ]:
#Frames actually processed by components
prcsdFrms_df = pd.DataFrame({
    "component":   pd.Series(dtype=int),
    "rcd_uS":      pd.Series(dtype=float),
    "frm_nm":      pd.Series(dtype=int),
    "frm_sz_b":    pd.Series(dtype=float),
    "cmp_ltnc_uS": pd.Series(dtype=float),
    "ntwrk_lt_uS": pd.Series(dtype=float),
    "snt_uS":      pd.Series(dtype=float),
    "done_uS":     pd.Series(dtype=float)
}) # contains no dropped/missed frames

#Frames that components were not ready to receive
drpmsdFrms_df = pd.DataFrame({
    "component":   pd.Series(dtype=int),
    "rcd_uS":      pd.Series(dtype=float),
    "frm_nm":      pd.Series(dtype=int),
    "frm_sz_b":    pd.Series(dtype=float),
    "lstDone_uS":  pd.Series(dtype=float)
}) # contains no dropped/missed frames

#Data on all sent frames
sentFrms_df = pd.DataFrame({
    "component":   pd.Series(dtype=int),
    "snt_uS":      pd.Series(dtype=float),
    "frm_nm":      pd.Series(dtype=int),
    "frm_sz_b":    pd.Series(dtype=float)
}) 

#set of all frame numbers from sender
all_frm_set = set(range(1, frame_cnt + 1))   # range is exclusive at the end, so add 1 for inclusive

# component 0 is the sender
clk_uS      = zeros = np.zeros(cmpnt_cnt+1, dtype=float) #Time last frame finished processing

vrbs = True

lf = open("output.txt", "w")

for f in range(0, frame_cnt):
    frm_sz0_b = B_b*fs_smpls_B[f]
    if vrbs: print(f"{clk_uS[0]} Send frame {f} Size: {frm_sz0_b:10.2f}", file=lf)
    row = (0,clk_uS[0],f,frm_sz0_b)
    sentFrms_df = pd.concat([sentFrms_df, pd.DataFrame([row], columns=sentFrms_df.columns)], ignore_index=True)
    for c in range(1, cmpnt_cnt+1):
        #set frame size to component output Size
        frm_szc_b = B_b*os_smpls_B[f*cmpnt_cnt+c]
        clk_c = clk_uS[c-1] #temp clk = upstream senders 'done/sent' value
        if c > 1:
            frm_sz_b = B_b*os_smpls_B[f*cmpnt_cnt+c-1]
        else:
            frm_sz_b = frm_sz0_b #component 1 uses sender frame size
        # component receives with network latency
        ntwrk_lt_uS = float(one_u*frm_sz_b/(G_1*nic_Gbps)) # fix this for jitter
        clk_c += ntwrk_lt_uS #Update temp clk for net latency
        rcd_uS = clk_c #Time would recv from upstream sender
        if vrbs: print(f"{clk_c} Component {c} Recv Frame {f} Size: {frm_sz_b:10.2f}", file=lf)
        if (clk_uS[c] > clk_c): 
            if vrbs: print(f"{clk_c} Component {c} Missed Frame {f}", file=lf)
            row = (c,rcd_uS,f,frm_sz_b,clk_uS[c])
            drpmsdFrms_df = pd.concat([drpmsdFrms_df, pd.DataFrame([row], columns=drpmsdFrms_df.columns)], ignore_index=True)
            break; #If temp clk > components last 'done' time, frame is missed for this and all downstream components
        # component processes with compute latency
        cmp_ltnc_uS = float(n_u*cmp_ltnc_nS_B*frm_sz_b*b_B) # fix this for jitter
        clk_c += cmp_ltnc_uS #Update temp clk for compute latency
        clk_c += 10 #add overhead
        snt_uS = clk_c
        row = (c,snt_uS,f,frm_szc_b)
        sentFrms_df = pd.concat([sentFrms_df, pd.DataFrame([row], columns=sentFrms_df.columns)], ignore_index=True)
        clk_c += 10 #add overhead
        clk_uS[c]  = clk_c #Set as last 'done' time
        if vrbs: print(f"{clk_c} Component {c} Done Frame {f} Size: {frm_sz_b:10.2f}", file=lf)
        #add prcsdFrms_df row
        row = (c,rcd_uS,f,frm_sz_b,cmp_ltnc_uS,ntwrk_lt_uS,snt_uS,clk_uS[c])
        prcsdFrms_df = pd.concat([prcsdFrms_df, pd.DataFrame([row], columns=prcsdFrms_df.columns)], ignore_index=True)
   # Sender Rate Sleep
    rtSlp_uS   = float(one_u*frm_sz0_b / (G_1*avg_bit_rt_Gbps))
    clk_uS[0] += rtSlp_uS

lf.close()   # must close manually
    

In [ ]:
for i in range(0, cmpnt_cnt + 1):

    sim_tm_uS = sentFrms_df.loc[sentFrms_df["component"] == i, "snt_uS"][1:].reset_index(drop=True)
    dts_S = u_1*(sentFrms_df.loc[sentFrms_df["component"] == i, "snt_uS"][1:].reset_index(drop=True) - sentFrms_df.loc[sentFrms_df["component"] == i, "snt_uS"][:-1].reset_index(drop=True))
    szs_b = sentFrms_df.loc[sentFrms_df["component"] == i, "frm_sz_b"][1:].reset_index(drop=True)

    btRt_Mbps = one_M*szs_b/dts_S
    # Display statistics
    btRt_stats_Mbps = compute_moments(btRt_Mbps)

    print(f"Component {i} Send Delta Statistics:")
    for k, v in btRt_stats_Mbps.items():
        print(f"{k}: {v:.3f}")

    # Plot
    plt.figure(figsize=(10, 4))
    plt.plot(u_1*sim_tm_uS/60, btRt_Mbps, marker='o', linestyle='-')
    plt.title(f"Component {i} Send bit Rate (Mbps)")
    plt.xlabel('Time (minutes)')
    plt.ylabel('Mbps')
    #plt.ylim(0, np.isfinite(recd['fps_roll'].mean()))
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # print(recd.tail(5))

    # recd.drop(recd.index, inplace=True)

    # print(recd.tail(5))

In [ ]:
# # Replace this with the actual path to your log file
# log_path = "output.txt"

# # Load and inspect
# lines = load_log_file(log_path)
# print(f"Loaded {len(lines)} lines from the log.")

# # Extract lines with frame rate information
# bit_rate_lines = [line for line in lines if "Send frame" in line]

# # Get frame rate values from timestamps
# frame_szs_b = []
# ts_uS = []
# for line in bit_rate_lines:
#     match = re.search(r"Size:  ([\d.]+)", line)
#     frame_szs_b.append(float(match.group(1) if match else None))
#     if match:
#         # Extract timestamp if available at the beginning of the line
#         ts_match = re.match(r"([\d.]+)", line)
#         ts_uS.append(float(ts_match.group(1) if ts_match else None))
#     else:
#         print("No match")

# # Create a DataFrame
# frame_rate_df = pd.DataFrame({
#     'ts_uS': ts_uS,
#     'frame_szs_b': frame_szs_b
# })

# sim_tm_uS = frame_rate_df["ts_uS"][1:].reset_index(drop=True)
# dts_S = u_1*(frame_rate_df["ts_uS"][1:].reset_index(drop=True) - frame_rate_df["ts_uS"][:-1].reset_index(drop=True))
# szs_b = frame_rate_df["frame_szs_b"][1:].reset_index(drop=True)

# btRt_Mbps = one_M*szs_b/dts_S
# # Display statistics
# btRt_stats_Mbps = compute_moments(btRt_Mbps)

# print(f"Sender Bit Rate Statistics:")
# for k, v in btRt_stats_Mbps.items():
#     print(f"{k}: {v:.3f}")

# # Plot
# plt.figure(figsize=(10, 4))
# plt.plot(u_1*sim_tm_uS/60, btRt_Mbps, marker='o', linestyle='-')
# plt.title(f"Sender Bit Rate (Mbps)")
# plt.xlabel('Time (minutes)')
# plt.ylabel('Mbps')
# #plt.ylim(0, np.isfinite(recd['fps_roll'].mean()))
# plt.grid(True)
# plt.tight_layout()
# plt.show()


In [ ]:
for i in range(1, cmpnt_cnt + 1):
    dt_S_arr           = np.diff(u_1*prcsdFrms_df.loc[prcsdFrms_df["component"] == i, "rcd_uS"])
    frame_rates_Hz_arr = 1/dt_S_arr

    # Display statistics
    frame_rate_stats = compute_moments(frame_rates_Hz_arr)

    print(f"Component {i} Frame Rate Statistics:")
    for k, v in frame_rate_stats.items():
        print(f"{k}: {v:.3f}")

    # Plot Frame Rate
    import numpy as np
    import seaborn as sns
    import matplotlib.pyplot as plt

    plt.figure(figsize=(8, 5))

    timestamps_S_arr = u_1*prcsdFrms_df.loc[prcsdFrms_df["component"] == i, "rcd_uS"][1:]

    sns.lineplot(x=timestamps_S_arr/60.0, y=frame_rates_Hz_arr, marker="o")       # line + markers
    # —or using pure matplotlib:
    # plt.plot(x, y, marker='o', linestyle='-')

    plt.title(f"Component {i} Recv Frame Rate (Hz)")
    plt.xlabel("Minutes")
    plt.ylabel("Hz")
    plt.tight_layout()
    plt.show()

    # plt.figure(figsize=(8, 5))
    # sns.lineplot(x=[u_1*float(x)/60.0 for x in done_timestamps_uS_lst[1:]], y=[u_m*(recd_timestamps_uS_lst[i+1] - done_timestamps_uS_lst[i]) for i in range(len(done_timestamps_uS_lst) - 1)], marker="o")       # line + markers
    # plt.title(f"Component {index} Recv Frame Delta (uS)")
    # plt.xlabel("Minutes")
    # plt.ylabel("mS")
    # plt.tight_layout()
    # plt.show()


In [ ]:
# for drop frames tracking - time delta between new recvd and last done (drop if negative)
# OK in your above data frame, i need to calculate all algebraic differences between the columns 'rcd_uS' - 'done_uS' such that the value of 'done_uS' 
# is obtained from the row previous to that of 'rcd_uS' unless the value of 'done_uS' in the same row as 'rcd_uS' is NaN in which case that row is skipped.  
# Under these conditions the next row will find the value of the previous rows value of 'done_uS' will be NaN and so the next more previous values 
# of 'done_uS' should be used to find the most recent non-NaN value for 'done_uS'.
# The first row for each component should be omitted from processing

for i in range(1, cmpnt_cnt + 1):

    plt.figure(figsize=(8, 5))
    # sns.lineplot(x=[u_1*float(x)/60.0 for x in done_timestamps_uS_lst[1:]], y=[u_m*(recd_timestamps_uS_lst[i+1] - done_timestamps_uS_lst[i]) for i in range(len(done_timestamps_uS_lst) - 1)], marker="o")       # line + markers
    # sns.lineplot(x=df.loc[df["component"] == i, "frm_nm"][1:], y=[u_m*(recd_timestamps_uS_lst[i+1] - done_timestamps_uS_lst[i]) for i in range(len(done_timestamps_uS_lst) - 1)], marker="o")       # line + markers

    rcd_uS =  prcsdFrms_df.loc[prcsdFrms_df["component"] == i,  "rcd_uS"][1:].reset_index(drop=True)
    done_uS = prcsdFrms_df.loc[prcsdFrms_df["component"] == i, "done_uS"][:-1].reset_index(drop=True)

    # print(i)
    # print(rcd)
    # print(done)

    delta_mS = u_m*(rcd_uS - done_uS)

    # Display statistics
    delta_stats_mS = compute_moments(delta_mS)

    print(f"Component {i} Recv Delta Statistics:")
    for k, v in delta_stats_mS.items():
        print(f"{k}: {v:.3f}")

    # print(diff)
    # print(df.loc[df["component"] == i, "frm_nm"][1:])
    
    plt.figure(figsize=(8, 5))
    sns.lineplot(x=prcsdFrms_df.loc[prcsdFrms_df["component"] == i, "frm_nm"][1:].reset_index(drop=True), y=delta_mS , marker="o")       # line + markers
    plt.title(f"Component {i} Recv Frame Delta")
#    plt.xlabel("Minutes")
    plt.xlabel("Frame")
    plt.ylabel("mS")
    plt.tight_layout()
    plt.show()



In [ ]:
for i in range(1, cmpnt_cnt + 1):

    sim_tm_uS = prcsdFrms_df.loc[prcsdFrms_df["component"] == i, "rcd_uS"][1:].reset_index(drop=True)
    dts_S = u_1*(prcsdFrms_df.loc[prcsdFrms_df["component"] == i, "rcd_uS"][1:].reset_index(drop=True) - prcsdFrms_df.loc[prcsdFrms_df["component"] == i, "rcd_uS"][:-1].reset_index(drop=True))
    szs_b = prcsdFrms_df.loc[prcsdFrms_df["component"] == i, "frm_sz_b"][1:].reset_index(drop=True)

    btRt_Mbps = one_M*szs_b/dts_S
    # Display statistics
    btRt_stats_Mbps = compute_moments(btRt_Mbps)

    print(f"Component {i} Recv Delta Statistics:")
    for k, v in btRt_stats_Mbps.items():
        print(f"{k}: {v:.3f}")

    # Plot
    plt.figure(figsize=(10, 4))
    plt.plot(u_1*sim_tm_uS/60, btRt_Mbps, marker='o', linestyle='-')
    plt.title(f"Component {i} Recv bit Rate (Mbps)")
    plt.xlabel('Time (minutes)')
    plt.ylabel('Mbps')
    #plt.ylim(0, np.isfinite(recd['fps_roll'].mean()))
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # print(recd.tail(5))

    # recd.drop(recd.index, inplace=True)

    # print(recd.tail(5))

In [ ]:
for i in range(1, cmpnt_cnt + 1):

    cmpLt_mS = u_m*prcsdFrms_df.loc[prcsdFrms_df["component"] == i, "cmp_ltnc_uS"]
    # Display statistics
    cmpLt_stats_mS = compute_moments(cmpLt_mS)

    print(f"Component {i} Recv Delta Statistics:")
    for k, v in cmpLt_stats_mS.items():
        print(f"{k}: {v:.3f}")

    # Plot
    plt.figure(figsize=(10, 4))
    plt.plot(u_1*prcsdFrms_df.loc[prcsdFrms_df["component"] == i, "rcd_uS"]/60, cmpLt_mS, marker='o', linestyle='-')
    plt.title(f"Component {i} Comp Latency (mS)")
    plt.xlabel('Time (minutes)')
    plt.ylabel('Latency (mS)')
    #plt.ylim(0, np.isfinite(recd['fps_roll'].mean()))
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # print(recd.tail(5))

    # recd.drop(recd.index, inplace=True)

    # print(recd.tail(5))

In [ ]:
for i in range(1, cmpnt_cnt + 1):

    ntwrkLt_uS = prcsdFrms_df.loc[prcsdFrms_df["component"] == i, "ntwrk_lt_uS"]
    # Display statistics
    ntwrkLt_stats_uS = compute_moments(ntwrkLt_uS)

    print(f"Component {i} Recv Delta Statistics:")
    for k, v in ntwrkLt_stats_uS.items():
        print(f"{k}: {v:.3f}")

    # Plot
    plt.figure(figsize=(10, 4))
    plt.plot(u_1*prcsdFrms_df.loc[prcsdFrms_df["component"] == i, "rcd_uS"]/60, ntwrkLt_uS, marker='o', linestyle='-')
    plt.title(f"Component {i} Network Latency (uS)")
    plt.xlabel('Time (minutes)')
    plt.ylabel('Latency (uS)')
    #plt.ylim(0, np.isfinite(recd['fps_roll'].mean()))
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # print(recd.tail(5))

    # recd.drop(recd.index, inplace=True)

    # print(recd.tail(5))

In [ ]:
for i in range(1, cmpnt_cnt + 1):
    
    # Plot
    plt.figure(figsize=(10, 4))
    plt.plot(u_1*prcsdFrms_df.loc[prcsdFrms_df["component"] == i, "rcd_uS"]/60, prcsdFrms_df.loc[prcsdFrms_df["component"] == i, "frm_nm"], marker='o', linestyle='-')
    plt.title(f"Component {i} Frame Recption Over Time")
    plt.xlabel('Time (Minutes)')
    plt.ylabel('recd (Frame num)')
    plt.ylim(0, max(prcsdFrms_df['frm_nm']) * 1.2)
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
for i in range(1, cmpnt_cnt + 1):
    # Plot
    plt.plot(u_1*prcsdFrms_df.loc[prcsdFrms_df["component"] == i, "rcd_uS"]/60, prcsdFrms_df.loc[prcsdFrms_df["component"] == i, "frm_nm"], marker='o', linestyle='-')
    
# Add labels and legend
plt.xlabel('Sim Time (Mins)')
plt.ylabel('Frame Number')
plt.title('Time Frame Recvd')
plt.legend()

# Show grid and plot
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
for i in range(1, cmpnt_cnt + 1):
    # Plot
    plt.plot(u_1*prcsdFrms_df.loc[prcsdFrms_df["component"] == i, "done_uS"]/60, prcsdFrms_df.loc[prcsdFrms_df["component"] == i, "frm_nm"], marker='o', linestyle='-')
    
# Add labels and legend
plt.xlabel('Sim Time (Hrs)')
plt.ylabel('Frame Number')
plt.title('Time Frame Done')
plt.legend()

# Show grid and plot
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
drpmsdFrms_df.head(5)

In [ ]:
drp_sets_by_component = drpmsdFrms_df.groupby("component")["frm_nm"].apply(set)
cmpnt_drp_nms = set(drpmsdFrms_df["component"].unique())
for c in cmpnt_drp_nms:
    print(f"Number of drops for component {c}: {len(drp_sets_by_component[c])}")

In [ ]:
drp_sets_by_component

In [ ]:
cmpnt_drp_nms

In [ ]:
prcsd_frm_sets_by_component = prcsdFrms_df.groupby("component")["frm_nm"].apply(set)
cmpnt_frn_nms = set(prcsdFrms_df["component"].unique())
for c in cmpnt_frn_nms:
    print(f"Number of processed frames for component {c}: {len(prcsd_frm_sets_by_component[c])}")

In [ ]:
drpdFrmsFrctn_df = pd.DataFrame({
    "component": pd.Series(dtype=int),
    "drp_frctn":   pd.Series(dtype=int)
})

cmpnt_drp_nms = set(drpmsdFrms_df["component"].unique())
#dataframe record for components with no drops
for c in set(range(1,cmpnt_cnt+1)) - cmpnt_drp_nms: #the set of compnents with no drops
    row = (c,0)
    drpdFrmsFrctn_df = pd.concat([drpdFrmsFrctn_df, pd.DataFrame([row], columns=drpdFrmsFrctn_df.columns)], ignore_index=True)
    
#dataframe record for components with drops
drp_sets_by_component = drpmsdFrms_df.groupby("component")["frm_nm"].apply(set)
for c in cmpnt_drp_nms:
    row = (c,len(drp_sets_by_component[c])/(len(drp_sets_by_component[c]) + len(prcsd_frm_sets_by_component[c])))
    drpdFrmsFrctn_df = pd.concat([drpdFrmsFrctn_df, pd.DataFrame([row], columns=drpdFrmsFrctn_df.columns)], ignore_index=True)


# drpdFrmsFrctn_df

# Plot
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import MaxNLocator

x = drpdFrmsFrctn_df['component'].astype(int)
y = drpdFrmsFrctn_df['drp_frctn']
max_y = np.nanmax(y) if len(y) else 1.0

fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(x, y, marker='o', linestyle='-')
ax.set_title("Component Drop Fraction")
ax.set_xlabel("Component")
ax.set_ylabel("Fraction")
# ax.set_ylim(-0.01, max_y * 1.2)

# Force integer tick locations
ax.xaxis.set_major_locator(MaxNLocator(integer=True))

ax.grid(True)
fig.tight_layout()
plt.show()

#Frames that components missed because they were never received
msdFrmsFrctn_df = pd.DataFrame({
    "component": pd.Series(dtype=int),
    "msd_frctn": pd.Series(dtype=int)
})

#Compnents miss frames when upstream senders do not send
#Upstrean components do not send frames they miss or drop
#The effect is cummulative for downstream components

#For each component, This equates to the number of sender frames - (number processd + the number droped (since they were not missed))

for c in range(1,cmpnt_cnt+1):
    # print(f"drpdFrmsFrctn_df for component {c}: {drpdFrmsFrctn_df.loc[drpdFrmsFrctn_df["component"] == (c-1), "drp_frctn"]}")
    # row = (c,1-(len(prcsd_frm_sets_by_component[c]) + len(set(drpmsdFrms_df.loc[drpmsdFrms_df["component"] == c, "frm_nm"])))/len(all_frm_set))
    row = (c,1-(len(set(prcsdFrms_df.loc[prcsdFrms_df["component"] == c, "frm_nm"])) + len(set(drpmsdFrms_df.loc[drpmsdFrms_df["component"] == c, "frm_nm"])))/len(all_frm_set))
    msdFrmsFrctn_df = pd.concat([msdFrmsFrctn_df, pd.DataFrame([row], columns=msdFrmsFrctn_df.columns)], ignore_index=True)

x = msdFrmsFrctn_df['component'].astype(int)
y = msdFrmsFrctn_df['msd_frctn']
ax_y = np.nanmax(y) if len(y) else 1.0

fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(x, y, marker='o', linestyle='-')
ax.set_title("Component Miss Fraction")
ax.set_xlabel("Component")
ax.set_ylabel("Fraction")
# ax.set_ylim(-0.01, max_y * 1.2)

# Force integer tick locations
ax.xaxis.set_major_locator(MaxNLocator(integer=True))

ax.grid(True)
fig.tight_layout()
plt.show()


In [ ]:
# for c in range(1,cmpnt_cnt+1):
#     print(len(set(drpmsdFrms_df.loc[drpmsdFrms_df["component"] == c, "frm_nm"])))